In [1]:
# Dependencies
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import time
from datetime import timedelta,datetime,date

# Graphing Dependencies
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Libraries to support access to the  Census API wrapper
from census import Census
from config import census_api_key as api_key
from config import beer_key
from config import g_web
from us import states

# Import Census Data

In [2]:
#setting an API object for the census data, with the desired year; 2016 is the latest data available at this level of detail from teh ACS5
c = Census(api_key, year = 2016)

#Setting the path for a sheet that contains codes and mapping for different categories of Census data
census_terms_path = ('Census_search_terms.xlsx')

#Creating dataframes from the XLSX which contain the codes for 59 different metrics and the FIPS state codes
census_terms_df = pd.read_excel(census_terms_path, sheetname = "Sheet1")
fips_codes_df = pd.read_excel(census_terms_path, sheetname = "Sheet2", dtype = str)

In [3]:
#grabbing census data from the Census API and putting into a list
census_data = []

for row in census_terms_df['code']:
    try:
        acs5_data = c.acs5.state(row, Census.ALL)
        census_data.append(acs5_data)
        print(f"grabbing {row}")
    except:
        print(f"grabbing {row} failed")

grabbing B01001_001E
grabbing B01001_002E
grabbing B01001_026E
grabbing B01002_001E
grabbing B01002_002E
grabbing B01002_003E
grabbing B23025_001E
grabbing B23025_002E
grabbing B23025_003E
grabbing B23025_004E
grabbing B23025_005E
grabbing B23025_006E
grabbing B23025_007E
grabbing B06009_001E
grabbing B06009_002E
grabbing B06009_003E
grabbing B06009_004E
grabbing B06009_005E
grabbing B06009_006E
grabbing B06010_001E
grabbing B06010_002E
grabbing B06010_003E
grabbing B06010_004E
grabbing B06010_005E
grabbing B06010_006E
grabbing B06010_007E
grabbing B06010_008E
grabbing B06010_009E
grabbing B06010_010E
grabbing B06010_011E
grabbing B08122_001E
grabbing B08122_005E
grabbing B08122_009E
grabbing B08122_013E
grabbing B08122_017E
grabbing B08122_021E
grabbing B08122_025E
grabbing B08124_001E
grabbing B08124_002E
grabbing B08124_003E
grabbing B08124_004E
grabbing B08124_005E
grabbing B08124_006E
grabbing B08124_007E
grabbing B08126_001E
grabbing B08126_002E
grabbing B08126_003E
grabbing B081

In [4]:
#taking all data from the list and adding it together into a single list
#then create a dataframe out of each and appending together

for i,data in enumerate(census_data):
    if i == 0:
        all_census_data_df = pd.DataFrame(data)
        all_census_data_df.set_index('state', inplace = True)
    else:
        all_census_data_df_2 = pd.DataFrame(data)
        all_census_data_df_2.set_index('state', inplace = True)
        all_census_data_df = all_census_data_df.merge(all_census_data_df_2, left_index = True, right_index = True)

In [5]:
#create a dictionary with variable keys as the census code and variables as the variable names
columns = dict(zip(census_terms_df['code'], census_terms_df['var_name']))

In [6]:
#rename columns to the human friendly variable names & then reset the index
all_census_data_df = all_census_data_df.rename(columns = columns)
all_census_data_df.reset_index(inplace = True)

#check out the dataset
all_census_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 60 columns):
state                                          52 non-null object
tot_pop                                        52 non-null object
male_pop                                       52 non-null float64
female_pop                                     52 non-null float64
median_age                                     52 non-null float64
median_age_male                                52 non-null float64
median_age_female                              52 non-null float64
employ_status_tot                              52 non-null float64
employ_status_working                          52 non-null float64
employ_status_working_civ                      52 non-null float64
employ_status_working_civ_employed             52 non-null float64
employ_status_working_civ_unemployed           52 non-null float64
employ_status_working_military                 52 non-null float64
employ_status_not_working   

In [7]:
#merge the state codes into the list so that we have a human readable State
all_census_data_df_with_states = pd.merge(all_census_data_df, fips_codes_df, left_on = 'state', right_on = "FIPS", how = "inner")

In [8]:
#Perform some cleansing on the file to remove unwanted columns and to ensure that one of population metrics is numeric data
all_census_data_df_with_states.drop('state', axis = 1, inplace = True)
all_census_data_df_with_states['tot_pop'] = all_census_data_df_with_states['tot_pop'].apply(pd.to_numeric)

#recheck the data
all_census_data_df_with_states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 51
Data columns (total 62 columns):
tot_pop                                        52 non-null int64
male_pop                                       52 non-null float64
female_pop                                     52 non-null float64
median_age                                     52 non-null float64
median_age_male                                52 non-null float64
median_age_female                              52 non-null float64
employ_status_tot                              52 non-null float64
employ_status_working                          52 non-null float64
employ_status_working_civ                      52 non-null float64
employ_status_working_civ_employed             52 non-null float64
employ_status_working_civ_unemployed           52 non-null float64
employ_status_working_military                 52 non-null float64
employ_status_not_working                      52 non-null float64
birth_ed_tot                

# Acquire breweries data

In [9]:
# API call for beermapping.com to get the data for breweries in each state
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]


base_url = "http://beermapping.com/webservice/locstate/"

beer_id = []
brewery_state = []
zipcode = []
brewery_name = []
brewery_city = []
brewery_type = []

# Iterating through the states list to return all the results for each state
# For responses that are for breweries, appending the lists to get the desired data
for state in states:
    url = base_url + beer_key + "/" + state  + "&s=json"
    print(url)
    state_data = requests.get(url).json()
    for response in state_data:
        if response["status"] == "Brewery":
            beer_id.append(response["id"])
            brewery_state.append(response["state"])
            zipcode.append(response["zip"])
            brewery_name.append(response["name"])
            brewery_city.append(response["city"])
            brewery_type.append(response["status"])

        
brewery_df = pd.DataFrame({"Brew Mapping Id": beer_id,
             "State": brewery_state,
             "Zipcode": zipcode,
             "Brewery Name": brewery_name,
             "City": brewery_city,
                "Type": brewery_type})

brewery_df.head()

http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/AL&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/AK&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/AZ&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/AR&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/CA&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/CO&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/CT&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/DC&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/DE&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/FL&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2ee5235d3f2d8c61989fe39/GA&s=json
http://beermapping.com/webservice/locstate/c0323ad9e2e

,Brew Mapping Id,Brewery Name,City,State,Type,Zipcode
0,15706,Avondale Brewing Company,Birmingham,AL,Brewery,35222
1,15004,Back Forty Beer Co,Gadsden,AL,Brewery,35901
2,20646,Band of Brothers Brewing Co,Tuscaloosa,AL,Brewery,35401
3,20843,Big Beach Brewing Company,Gulf Shores,AL,Brewery,36542
4,18433,Black Warrior Brewing,Tuscaloosa,AL,Brewery,35401


In [10]:
# Clean some of the state abbreviations in the dataset
for ix,row in brewery_df.iterrows():
    if row["State"] == "Mi":
        brewery_df.at[ix,"State"]="MI"
    elif row ["State"] == "tx":
        brewery_df.at[ix,"State"]="TX"
    else: pass

In [11]:
# Export data to csv to investigate the data
brewery_df.to_csv('breweries_df.csv', index=False)

In [12]:
# Creating groups by state to integrate with census data
state_groups = brewery_df.groupby(["State"])
brew_state_count = state_groups["Brew Mapping Id"].count()
brew_state_count_df = pd.DataFrame({"brewery_count": brew_state_count})
brew_state_count_df.reset_index(inplace=True)

In [13]:
brew_state_count_df.rename(columns = {"State": "Abbrev"}, inplace = True)
brew_state_count_df.head()

,Abbrev,brewery_count
0,AK,19
1,AL,22
2,AR,14
3,AZ,37
4,CA,315


In [14]:
fips_codes_df.head()

,FIPS,State,Abbrev
0,01,Alabama,AL
1,02,Alaska,AK
2,04,Arizona,AZ
3,05,Arkansas,AR
4,06,California,CA


In [15]:
# Merge the FIPS data into the dataframe
brew_state_count_df = pd.merge(brew_state_count_df, fips_codes_df, how = "left", on = "Abbrev")
brew_state_count_df

,Abbrev,brewery_count,FIPS,State
0,AK,19,02,Alaska
1,AL,22,01,Alabama
2,AR,14,05,Arkansas
3,AZ,37,04,Arizona
4,CA,315,06,California
5,CO,200,08,Colorado
6,CT,22,09,Connecticut
7,DC,7,11,District of Columbia
8,DE,11,10,Delaware
9,FL,106,12,Florida


In [16]:
# Export to csv and to excel to check
brew_state_count_df.to_excel('breweries_state.xlsx', sheet_name = "count", index=False)
brew_state_count_df.to_csv('breweries_state.csv', index=False)

# Merge breweries data with census data

In [17]:
all_census_data_df_with_states.head()

,tot_pop,male_pop,female_pop,median_age,median_age_male,median_age_female,employ_status_tot,employ_status_working,employ_status_working_civ,employ_status_working_civ_employed,...,industry_finance_insure_realestate,industry_professional_scientific_management,industry_education_health_care,industry_arts_entertatinment,industry_other_services,industry_public_admin,industry_military,FIPS,State,Abbrev
0,4841164,2346193.0,2494971.0,38.6,37.1,40.0,3864302.0,2238654.0,2226504.0,2042025.0,...,112728.0,186890.0,447613.0,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL
1,736855,385296.0,351559.0,33.6,33.3,34.0,568732.0,400475.0,384093.0,353954.0,...,13312.0,29311.0,80448.0,31506.0,14685.0,40037.0,16339.0,02,Alaska,AK
2,6728577,3344106.0,3384471.0,37.1,35.8,38.4,5290839.0,3146717.0,3129344.0,2879372.0,...,231382.0,339764.0,621371.0,307084.0,140000.0,148691.0,17330.0,04,Arizona,AZ
3,2968472,1456694.0,1511778.0,37.7,36.4,39.2,2339307.0,1364187.0,1359742.0,1266552.0,...,59520.0,89990.0,301561.0,100604.0,60600.0,57238.0,4445.0,05,Arkansas,AR
4,38654206,19200970.0,19453236.0,36.0,34.9,37.1,30565746.0,19391320.0,19260868.0,17577142.0,...,1064127.0,2236699.0,3532774.0,1768887.0,913338.0,756478.0,129333.0,06,California,CA


In [18]:
#merge the breweries count into a dataframe
census_and_breweries = pd.merge(all_census_data_df_with_states, brew_state_count_df, how = 'left', on = ["FIPS", "State", "Abbrev"])
census_and_breweries.head()

,tot_pop,male_pop,female_pop,median_age,median_age_male,median_age_female,employ_status_tot,employ_status_working,employ_status_working_civ,employ_status_working_civ_employed,...,industry_professional_scientific_management,industry_education_health_care,industry_arts_entertatinment,industry_other_services,industry_public_admin,industry_military,FIPS,State,Abbrev,brewery_count
0,4841164,2346193.0,2494971.0,38.6,37.1,40.0,3864302.0,2238654.0,2226504.0,2042025.0,...,186890.0,447613.0,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL,22.0
1,736855,385296.0,351559.0,33.6,33.3,34.0,568732.0,400475.0,384093.0,353954.0,...,29311.0,80448.0,31506.0,14685.0,40037.0,16339.0,02,Alaska,AK,19.0
2,6728577,3344106.0,3384471.0,37.1,35.8,38.4,5290839.0,3146717.0,3129344.0,2879372.0,...,339764.0,621371.0,307084.0,140000.0,148691.0,17330.0,04,Arizona,AZ,37.0
3,2968472,1456694.0,1511778.0,37.7,36.4,39.2,2339307.0,1364187.0,1359742.0,1266552.0,...,89990.0,301561.0,100604.0,60600.0,57238.0,4445.0,05,Arkansas,AR,14.0
4,38654206,19200970.0,19453236.0,36.0,34.9,37.1,30565746.0,19391320.0,19260868.0,17577142.0,...,2236699.0,3532774.0,1768887.0,913338.0,756478.0,129333.0,06,California,CA,315.0


In [19]:
#export to excel to check
census_and_breweries.to_excel('census_and_breweries_state.xlsx', sheet_name = 'data')

## Merge census & brewery data with Winery Data

In [20]:
# reading the bonded wine producers data into a CSV
wineries_path = ('bonded-wine-producers-by-state-2017.csv')
wineries_df = pd.read_csv(wineries_path)
wineries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
State(abbrev)    50 non-null object
State            50 non-null object
winery_count     50 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


In [21]:
wineries_df.rename(columns = {'State(abbrev)': "Abbrev"}, inplace = True)
wineries_df.head()

,Abbrev,State,winery_count
0,AK,Alaska,8
1,AL,Alabama,35
2,AR,Arkansas,33
3,AZ,Arizona,94
4,CA,California,4816


In [22]:
#merge the wineries count into the dataframe
census_breweries_wineries = pd.merge(census_and_breweries, wineries_df, how = 'left', on = ["State", "Abbrev"])
len(census_breweries_wineries)

52

In [23]:
census_breweries_wineries.head()

,tot_pop,male_pop,female_pop,median_age,median_age_male,median_age_female,employ_status_tot,employ_status_working,employ_status_working_civ,employ_status_working_civ_employed,...,industry_education_health_care,industry_arts_entertatinment,industry_other_services,industry_public_admin,industry_military,FIPS,State,Abbrev,brewery_count,winery_count
0,4841164,2346193.0,2494971.0,38.6,37.1,40.0,3864302.0,2238654.0,2226504.0,2042025.0,...,447613.0,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL,22.0,35.0
1,736855,385296.0,351559.0,33.6,33.3,34.0,568732.0,400475.0,384093.0,353954.0,...,80448.0,31506.0,14685.0,40037.0,16339.0,02,Alaska,AK,19.0,8.0
2,6728577,3344106.0,3384471.0,37.1,35.8,38.4,5290839.0,3146717.0,3129344.0,2879372.0,...,621371.0,307084.0,140000.0,148691.0,17330.0,04,Arizona,AZ,37.0,94.0
3,2968472,1456694.0,1511778.0,37.7,36.4,39.2,2339307.0,1364187.0,1359742.0,1266552.0,...,301561.0,100604.0,60600.0,57238.0,4445.0,05,Arkansas,AR,14.0,33.0
4,38654206,19200970.0,19453236.0,36.0,34.9,37.1,30565746.0,19391320.0,19260868.0,17577142.0,...,3532774.0,1768887.0,913338.0,756478.0,129333.0,06,California,CA,315.0,4816.0


In [24]:
#export data to an excel document to check
census_breweries_wineries.to_excel('census_breweries_wineries_state.xlsx', sheet_name = 'data')

## Merge Census & brewery data with distillery data

In [25]:
#read distilleries document into a dataframe
distillery_path = ('operating-craft-distilleries-us-2016-by-state.csv')
distillery_df = pd.read_csv(distillery_path)
distillery_df.head()

,State,craft_distillery_count
0,California,118
1,New York,104
2,Washington,103
3,Colorado,72
4,Texas,71


In [26]:
#merge the distilleries data into the dataframe containing census data, brewery data and winery data
census_breweries_wineries_distilleries = pd.merge(census_breweries_wineries, distillery_df, how = 'left', on = "State")
census_breweries_wineries_distilleries.head()

,tot_pop,male_pop,female_pop,median_age,median_age_male,median_age_female,employ_status_tot,employ_status_working,employ_status_working_civ,employ_status_working_civ_employed,...,industry_arts_entertatinment,industry_other_services,industry_public_admin,industry_military,FIPS,State,Abbrev,brewery_count,winery_count,craft_distillery_count
0,4841164,2346193.0,2494971.0,38.6,37.1,40.0,3864302.0,2238654.0,2226504.0,2042025.0,...,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL,22.0,35.0,8.0
1,736855,385296.0,351559.0,33.6,33.3,34.0,568732.0,400475.0,384093.0,353954.0,...,31506.0,14685.0,40037.0,16339.0,02,Alaska,AK,19.0,8.0,6.0
2,6728577,3344106.0,3384471.0,37.1,35.8,38.4,5290839.0,3146717.0,3129344.0,2879372.0,...,307084.0,140000.0,148691.0,17330.0,04,Arizona,AZ,37.0,94.0,19.0
3,2968472,1456694.0,1511778.0,37.7,36.4,39.2,2339307.0,1364187.0,1359742.0,1266552.0,...,100604.0,60600.0,57238.0,4445.0,05,Arkansas,AR,14.0,33.0,3.0
4,38654206,19200970.0,19453236.0,36.0,34.9,37.1,30565746.0,19391320.0,19260868.0,17577142.0,...,1768887.0,913338.0,756478.0,129333.0,06,California,CA,315.0,4816.0,118.0


In [27]:
census_breweries_wineries_distilleries.replace(np.NaN, 0, inplace = True)

In [28]:
census_breweries_wineries_distilleries['tot_k_pop'] = census_breweries_wineries_distilleries['tot_pop'] / 1000

In [29]:
census_breweries_wineries_distilleries['breweries_k_pop'] = census_breweries_wineries_distilleries['brewery_count'] / census_breweries_wineries_distilleries['tot_k_pop']
census_breweries_wineries_distilleries['wineries_k_pop'] = census_breweries_wineries_distilleries['winery_count'] / census_breweries_wineries_distilleries['tot_k_pop']
census_breweries_wineries_distilleries['distilleries_k_pop'] = census_breweries_wineries_distilleries['craft_distillery_count'] / census_breweries_wineries_distilleries['tot_k_pop']
census_breweries_wineries_distilleries.head()

,tot_pop,male_pop,female_pop,median_age,median_age_male,median_age_female,employ_status_tot,employ_status_working,employ_status_working_civ,employ_status_working_civ_employed,...,FIPS,State,Abbrev,brewery_count,winery_count,craft_distillery_count,tot_k_pop,breweries_k_pop,wineries_k_pop,distilleries_k_pop
0,4841164,2346193.0,2494971.0,38.6,37.1,40.0,3864302.0,2238654.0,2226504.0,2042025.0,...,01,Alabama,AL,22.0,35.0,8.0,4841.164,0.004544,0.007230,0.001652
1,736855,385296.0,351559.0,33.6,33.3,34.0,568732.0,400475.0,384093.0,353954.0,...,02,Alaska,AK,19.0,8.0,6.0,736.855,0.025785,0.010857,0.008143
2,6728577,3344106.0,3384471.0,37.1,35.8,38.4,5290839.0,3146717.0,3129344.0,2879372.0,...,04,Arizona,AZ,37.0,94.0,19.0,6728.577,0.005499,0.013970,0.002824
3,2968472,1456694.0,1511778.0,37.7,36.4,39.2,2339307.0,1364187.0,1359742.0,1266552.0,...,05,Arkansas,AR,14.0,33.0,3.0,2968.472,0.004716,0.011117,0.001011
4,38654206,19200970.0,19453236.0,36.0,34.9,37.1,30565746.0,19391320.0,19260868.0,17577142.0,...,06,California,CA,315.0,4816.0,118.0,38654.206,0.008149,0.124592,0.003053


In [30]:
#Export dataframe to a excel
census_breweries_wineries_distilleries.to_excel('census_breweries_wineries_distilleries_state.xlsx', sheet_name = 'data')

## Adding the Google data

In [31]:
#Creating the search terms for the GMAPs API
region_id = "US"

#group search term lists
winery_l=["winery","vineyard","wine+spirits","wine+garden"]
distillery_l=["distillery","distill+spirit","distiller"]
brewery_l=["brewery","brew+pub","taphouse","beer+garden"]

#combined lists
term_search= winery_l+distillery_l+brewery_l

In [32]:
#Create empty lists
name_data=[]
address_data=[]
lon_data=[]
lat_data=[]
place_id=[]
json_urls=[]
rating=[]
state_abr=[]
place_search=[]
# set up a parameters dictionary

# base url
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

for items in term_search:
    start_time=time.time()
    print("Query terms: "+items)
    for state in states:
        params = {"key": g_web,"query": items +"+"+state+"+"+region_id}
        response = requests.get(base_url, params=params)
        place_info=response.json()
        for res in place_info["results"]:
            name_data.append(res["name"])
            address_data.append(res["formatted_address"])
            lon_data.append(res["geometry"]["location"]["lng"])
            lat_data.append(res["geometry"]["location"]["lat"])
            place_id.append(res["place_id"])
            state_abr.append(state)
            place_search.append(items)
            json_urls.append(response.url)
        #time.sleep(1.5)
    print("        API DATA RETRIEVAL COMPLETE for search term: %s. (elapsed time: %s seconds)" %((items), round(time.time()-start_time,3)))

Query terms: winery
        API DATA RETRIEVAL COMPLETE for search term: winery. (elapsed time: 38.585 seconds)
Query terms: vineyard
        API DATA RETRIEVAL COMPLETE for search term: vineyard. (elapsed time: 35.33 seconds)
Query terms: wine+spirits
        API DATA RETRIEVAL COMPLETE for search term: wine+spirits. (elapsed time: 36.534 seconds)
Query terms: wine+garden
        API DATA RETRIEVAL COMPLETE for search term: wine+garden. (elapsed time: 28.667 seconds)
Query terms: distillery
        API DATA RETRIEVAL COMPLETE for search term: distillery. (elapsed time: 31.305 seconds)
Query terms: distill+spirit
        API DATA RETRIEVAL COMPLETE for search term: distill+spirit. (elapsed time: 28.821 seconds)
Query terms: distiller
        API DATA RETRIEVAL COMPLETE for search term: distiller. (elapsed time: 30.03 seconds)
Query terms: brewery
        API DATA RETRIEVAL COMPLETE for search term: brewery. (elapsed time: 41.987 seconds)
Query terms: brew+pub
        API DATA RETRIEVAL

In [33]:
# joins all lists into one dataframe
data_output = pd.DataFrame(np.column_stack([place_search,name_data,address_data,state_abr,lon_data,lat_data,place_id,json_urls]),
                       columns=["query","name","address","state","lon","lat","place_id","response_url"])

In [34]:
data_output["establishment"] = data_output["query"]

In [35]:
# Creates a category by establishment, replacing search term by the lists
for term in winery_l:
    for i in range(data_output.establishment.count()):
        data_output.establishment.i= data_output["establishment"].replace(
        to_replace=term,
        value="winery",
        inplace=True
        )

for term in brewery_l:
    for i in range(data_output.establishment.count()):
        data_output.establishment.i= data_output["establishment"].replace(
        to_replace=term,
        value="brewery",
        inplace=True
        )
        
for term in distillery_l:
    for i in range(data_output.establishment.count()):
        data_output.establishment.i= data_output["establishment"].replace(
        to_replace=term,
        value="distillery",
        inplace=True
        )

In [39]:
#cleanses the addresses and find zip codes
data_output["address"]=data_output["address"].str.replace("United States","USA")

In [40]:
#cleanses based on place_ids
data_output=data_output.drop_duplicates(subset="place_id")
data_output.set_index("place_id")

,query,name,address,state,lon,lat,response_url,establishment
place_id,,,,,,,,
ChIJ_-ZwGslGiogRNG4w72l_cJ0,winery,Maraella Winery and Vineyard,"5296 Old Us Highway 278 E, Hokes Bluff, AL 359...",AL,-85.872436,33.9873,https://maps.googleapis.com/maps/api/place/tex...,winery
ChIJKfFanqv8iogRKzY6loUAOK4,winery,Fruithurst Winery Co,"27091 Co Rd 49, Fruithurst, AL 36262, USA",AL,-85.440266,33.797355,https://maps.googleapis.com/maps/api/place/tex...,winery
ChIJqePfQqwviYgRRf0JiuZ9xZc,winery,Corbin Farms Winery,"800 County Road 87, Calera, AL 35040, USA",AL,-86.7751933,33.1757435,https://maps.googleapis.com/maps/api/place/tex...,winery
ChIJ8SLierAAi4gRpZfWFBKt_io,winery,High Country Cellars,"849 Almon St, Heflin, AL 36264, USA",AL,-85.58999469999999,33.639259,https://maps.googleapis.com/maps/api/place/tex...,winery
ChIJ_zUL4jsUiogROBKFxirX9nM,winery,Wills Creek Winery,"10522 Duck Springs Rd, Attalla, AL 35954, USA",AL,-85.99668299999999,34.14828,https://maps.googleapis.com/maps/api/place/tex...,winery
ChIJ_5IwKgmRjogR4Wk1hcrugDk,winery,Queen Meats And Wine,"207 US-82, Prattville, AL 36067, USA",AL,-86.47600899999999,32.438956,https://maps.googleapis.com/maps/api/place/tex...,winery
ChIJ_3CMeJCri4gRmy_n94bLaHw,winery,White Oak Vineyards,"1484 Dry Hollow Rd, Anniston, AL 36207, USA",AL,-85.67219999999999,33.665,https://maps.googleapis.com/maps/api/place/tex...,winery
ChIJEbe2rEAyiYgRnoYKo4AbvWc,winery,Ozan Winery,"173 Co Rd 301, Calera, AL 35040, USA",AL,-86.71651,33.102898,https://maps.googleapis.com/maps/api/place/tex...,winery
ChIJYTNHovCEmogRqKm_G7gxPVo,winery,Perdido Vineyards,"22100 Co Rd 47, Perdido, AL 36562, USA",AL,-87.64802139999999,31.0194443,https://maps.googleapis.com/maps/api/place/tex...,winery


In [41]:
len(data_output)

4993

In [46]:
establishment_types = data_output['establishment'].unique()

In [76]:
data_output_grouped = pd.DataFrame(data_output.groupby(by = ['establishment', 'state']).count())
data_output_grouped.drop(['name', 'address', 'lon', 'lat', 'place_id', 'response_url'], axis = 1, inplace = True)
clean_data_output = data_output_grouped.loc['brewery']
clean_data_output.rename(columns = {"query": "google_brewery_count"}, inplace = True)
clean_data_output = clean_data_output.add(data_output_grouped.loc['winery'], fill_value=0)
clean_data_output.rename(columns = {"query": "google_winery_count"}, inplace = True)
clean_data_output = clean_data_output.add(data_output_grouped.loc['distillery'], fill_value = 0)
clean_data_output.rename(columns = {"query": "google_distillery_count"}, inplace = True)
clean_data_output.head()

,google_brewery_count,google_winery_count,google_distillery_count
state,,,
AK,34.0,14.0,8.0
AL,57.0,34.0,4.0
AR,46.0,39.0,2.0
AZ,52.0,43.0,12.0
CA,59.0,41.0,29.0


In [77]:
census_breweries_wineries_distilleries.head()

,tot_pop,male_pop,female_pop,median_age,median_age_male,median_age_female,employ_status_tot,employ_status_working,employ_status_working_civ,employ_status_working_civ_employed,...,FIPS,State,Abbrev,brewery_count,winery_count,craft_distillery_count,tot_k_pop,breweries_k_pop,wineries_k_pop,distilleries_k_pop
0,4841164,2346193.0,2494971.0,38.6,37.1,40.0,3864302.0,2238654.0,2226504.0,2042025.0,...,01,Alabama,AL,22.0,35.0,8.0,4841.164,0.004544,0.007230,0.001652
1,736855,385296.0,351559.0,33.6,33.3,34.0,568732.0,400475.0,384093.0,353954.0,...,02,Alaska,AK,19.0,8.0,6.0,736.855,0.025785,0.010857,0.008143
2,6728577,3344106.0,3384471.0,37.1,35.8,38.4,5290839.0,3146717.0,3129344.0,2879372.0,...,04,Arizona,AZ,37.0,94.0,19.0,6728.577,0.005499,0.013970,0.002824
3,2968472,1456694.0,1511778.0,37.7,36.4,39.2,2339307.0,1364187.0,1359742.0,1266552.0,...,05,Arkansas,AR,14.0,33.0,3.0,2968.472,0.004716,0.011117,0.001011
4,38654206,19200970.0,19453236.0,36.0,34.9,37.1,30565746.0,19391320.0,19260868.0,17577142.0,...,06,California,CA,315.0,4816.0,118.0,38654.206,0.008149,0.124592,0.003053


In [78]:
all_data_df = pd.merge(census_breweries_wineries_distilleries, clean_data_output, how = "left", left_on = "Abbrev", right_index = True)

In [79]:
all_data_df.head()

,tot_pop,male_pop,female_pop,median_age,median_age_male,median_age_female,employ_status_tot,employ_status_working,employ_status_working_civ,employ_status_working_civ_employed,...,brewery_count,winery_count,craft_distillery_count,tot_k_pop,breweries_k_pop,wineries_k_pop,distilleries_k_pop,google_brewery_count,google_winery_count,google_distillery_count
0,4841164,2346193.0,2494971.0,38.6,37.1,40.0,3864302.0,2238654.0,2226504.0,2042025.0,...,22.0,35.0,8.0,4841.164,0.004544,0.007230,0.001652,57.0,34.0,4.0
1,736855,385296.0,351559.0,33.6,33.3,34.0,568732.0,400475.0,384093.0,353954.0,...,19.0,8.0,6.0,736.855,0.025785,0.010857,0.008143,34.0,14.0,8.0
2,6728577,3344106.0,3384471.0,37.1,35.8,38.4,5290839.0,3146717.0,3129344.0,2879372.0,...,37.0,94.0,19.0,6728.577,0.005499,0.013970,0.002824,52.0,43.0,12.0
3,2968472,1456694.0,1511778.0,37.7,36.4,39.2,2339307.0,1364187.0,1359742.0,1266552.0,...,14.0,33.0,3.0,2968.472,0.004716,0.011117,0.001011,46.0,39.0,2.0
4,38654206,19200970.0,19453236.0,36.0,34.9,37.1,30565746.0,19391320.0,19260868.0,17577142.0,...,315.0,4816.0,118.0,38654.206,0.008149,0.124592,0.003053,59.0,41.0,29.0


In [80]:
all_data_df.to_excel("all_data.xls", sheet_name = "data")